In [1]:
import os
import math
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.transforms.autoaugment as autoaug
import torch.cuda.amp as amp
from timm.models.layers import DropPath
from tqdm import tqdm

# ----------------- Setup -----------------
def seed_everything(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ----------------- Dataset -----------------
class GeoAngleDataset(Dataset):
    def __init__(self, df, img_dir, transform):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(os.path.join(self.img_dir, row.filename)).convert('RGB')
        img = self.transform(img)
        angle = float(row.angle) % 360
        rad   = angle * math.pi / 180.0
        return img, torch.tensor([math.cos(rad), math.sin(rad)], dtype=torch.float32)

# ----------------- Model -----------------
class AngleModel(nn.Module):
    def __init__(self, dropout_p=0.1, drop_path_rate=0.1):
        super().__init__()
        # DINOv2 ViT-B/14 backbone
        self.backbone = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14', pretrained=True)

        # Add stochastic depth
        for blk in self.backbone.blocks:
            blk.drop_path = DropPath(drop_path_rate) if drop_path_rate > 0 else nn.Identity()

        # Freeze patch embedding & first half of blocks
        self.backbone.patch_embed.requires_grad_(False)
        num_blocks = len(self.backbone.blocks)
        for blk in self.backbone.blocks[: num_blocks // 2]:
            for p in blk.parameters():
                p.requires_grad = False

        # Regression head
        feat_dim = self.backbone.embed_dim
        self.dropout = nn.Dropout(dropout_p)
        self.head = nn.Sequential(
            nn.Linear(feat_dim, feat_dim // 2),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_p),
            nn.Linear(feat_dim // 2, 2),
        )

    def forward(self, x):
        feats = self.backbone(x)           # [B, C]
        feats = self.dropout(feats)
        out   = self.head(feats)           # [B, 2]
        # normalize to unit circle
        norm  = out.norm(dim=1, keepdim=True).clamp(min=1e-6)
        return out / norm

# ----------------- Metrics -----------------
def angle_mae(pred, tgt):
    dot  = (pred * tgt).sum(dim=1).clamp(-1, 1)
    diff = torch.acos(dot)  # radians
    return (diff * 180 / math.pi).mean().item()

def validate(model, loader):
    model.eval()
    maes = []
    with torch.no_grad():
        for imgs, tgt in loader:
            imgs, tgt = imgs.to(device), tgt.to(device)
            p = model(imgs)
            maes.append(angle_mae(p, tgt))
    return float(np.mean(maes))

# ----------------- Main -----------------
if __name__ == '__main__':
    DATA_ROOT  = ''
    DATA_ROOT2 = ''
    
    train_df = pd.read_csv(os.path.join(DATA_ROOT2, 'labels_train_updated.csv')).dropna(subset=['angle'])
    val_df   = pd.read_csv(os.path.join(DATA_ROOT,  'labels_val_updated.csv')).dropna(subset=['angle'])
    train_dir = os.path.join(DATA_ROOT, 'images_train')
    val_dir   = os.path.join(DATA_ROOT, 'images_val')

    # Ensure size is multiple of 14: 476 = 14*34
    IMG_SIZE = 476

    train_tf = T.Compose([
        T.RandomResizedCrop(IMG_SIZE, scale=(0.5, 1.0)),
        autoaug.RandAugment(),
        T.ColorJitter(0.3, 0.3, 0.3),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406],
                    [0.229, 0.224, 0.225]),
    ])
    val_tf = T.Compose([
        T.Resize(IMG_SIZE + 20),
        T.CenterCrop(IMG_SIZE),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406],
                    [0.229, 0.224, 0.225]),
    ])

    train_ds = GeoAngleDataset(train_df, train_dir, train_tf)
    val_ds   = GeoAngleDataset(val_df,   val_dir,   val_tf)
    train_loader = DataLoader(train_ds, batch_size=2, shuffle=True,
                              num_workers=0, pin_memory=True)
    val_loader   = DataLoader(val_ds,   batch_size=2, shuffle=False,
                              num_workers=0, pin_memory=True)

    model = AngleModel(dropout_p=0.1, drop_path_rate=0.1).to(device)

    # Optimizer: AdamW with separate LRs increased by 5×
    backbone_params = [p for p in model.backbone.parameters() if p.requires_grad]
    head_params     = list(model.head.parameters()) + list(model.dropout.parameters())
    optimizer = optim.AdamW([
        {"params": backbone_params, "lr": 5e-6},
        {"params": head_params,     "lr": 5e-4}
    ], weight_decay=1e-2)

    # Cosine LR schedule
    num_epochs = 30
    scheduler = optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=num_epochs, eta_min=1e-7
    )

    scaler = amp.GradScaler()
    best_mae = float('inf')
    accum_steps = 2  # to simulate batch size 16

    for epoch in range(1, num_epochs + 1):
        model.train()
        optimizer.zero_grad()
        running_loss = 0.0
        running_mae  = 0.0
        running_n    = 0

        pbar = tqdm(train_loader, desc=f"Epoch {epoch}")
        for i, (imgs, tgt) in enumerate(pbar, 1):
            imgs, tgt = imgs.to(device), tgt.to(device)
            with amp.autocast():
                out   = model(imgs)
                loss  = nn.MSELoss()(out, tgt) / accum_steps
            scaler.scale(loss).backward()

            # compute batch MAE on mixed-precision output
            with torch.no_grad():
                batch_mae = angle_mae(out.detach(), tgt)

            # accumulate stats
            running_loss += loss.item() * accum_steps
            running_mae  += batch_mae * imgs.size(0)
            running_n    += imgs.size(0)

            if i % accum_steps == 0:
                scaler.unscale_(optimizer)
                nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                scheduler.step()

            epoch_loss = running_loss / running_n
            epoch_mae  = running_mae / running_n
            pbar.set_postfix(train_loss=f"{epoch_loss:.4f}", train_mae=f"{epoch_mae:.2f}°")

        val_mae = validate(model, val_loader)
        print(f"Epoch {epoch} → Val MAE = {val_mae:.2f}°   (best {best_mae:.2f}°)")
        if val_mae < best_mae:
            best_mae = val_mae
            torch.save(model.state_dict(), 'best_model.pth')
            print("  ✔ New best model saved.")

    print(f"Training complete. Best Val MAE = {best_mae:.2f}°")


C:\Users\91997\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Using device: cuda


Using cache found in C:\Users\91997/.cache\torch\hub\facebookresearch_dinov2_main
C:\Users\91997/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\91997/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\91997/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
C:\Users\91997\AppData\Local\Temp\ipykernel_30652\3384266339.py:150: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()
Epoch 1:   0%|          | 0/3238 [00:00<?, ?it/s]C:\Users\91997\AppData\Local\Temp\ipykernel_30652\3384266339.p

Epoch 1 → Val MAE = 49.59°   (best inf°)
  ✔ New best model saved.


Epoch 2: 100%|██████████| 3238/3238 [20:08<00:00,  2.68it/s, train_loss=0.2081, train_mae=46.31°]


...

Epoch 28 → Val MAE = 20.56°   (best 20.14°)


Epoch 29: 100%|██████████| 3238/3238 [19:41<00:00,  2.74it/s, train_loss=0.0155, train_mae=8.57°]


Epoch 29 → Val MAE = 20.81°   (best 20.14°)


Epoch 30: 100%|██████████| 3238/3238 [19:41<00:00,  2.74it/s, train_loss=0.0147, train_mae=8.37°]


Epoch 30 → Val MAE = 21.32°   (best 20.14°)
Training complete. Best Val MAE = 20.14°


In [ ]:
# ----------------- Continue Training -----------------
additional_epochs = 20
start_epoch = num_epochs + 1
end_epoch   = num_epochs + additional_epochs

for epoch in range(start_epoch, end_epoch + 1):
    model.train()
    optimizer.zero_grad()
    running_loss = 0.0
    running_mae  = 0.0
    running_n    = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch}")
    for i, (imgs, tgt) in enumerate(pbar, 1):
        imgs, tgt = imgs.to(device), tgt.to(device)
        with amp.autocast():
            out  = model(imgs)
            loss = nn.MSELoss()(out, tgt) / accum_steps
        scaler.scale(loss).backward()

        # compute batch MAE
        with torch.no_grad():
            batch_mae = angle_mae(out.detach(), tgt)

        running_loss += loss.item() * accum_steps
        running_mae  += batch_mae * imgs.size(0)
        running_n    += imgs.size(0)

        if i % accum_steps == 0:
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()

        epoch_loss = running_loss / running_n
        epoch_mae  = running_mae / running_n
        pbar.set_postfix(train_loss=f"{epoch_loss:.4f}", train_mae=f"{epoch_mae:.2f}°")

    val_mae = validate(model, val_loader)
    print(f"Epoch {epoch} → Val MAE = {val_mae:.2f}°   (best {best_mae:.2f}°)")
    if val_mae < best_mae:
        best_mae = val_mae
        torch.save(model.state_dict(), 'best_model.pth')
        print("  ✔ New best model saved.")

print(f"Continued training complete. Best Val MAE = {best_mae:.2f}°")


Epoch 31:   0%|          | 0/3238 [00:00<?, ?it/s]C:\Users\91997\AppData\Local\Temp\ipykernel_30652\4116418947.py:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
Epoch 31: 100%|██████████| 3238/3238 [20:04<00:00,  2.69it/s, train_loss=0.0125, train_mae=7.86°]


Epoch 31 → Val MAE = 20.46°   (best 20.14°)


Epoch 32: 100%|██████████| 3238/3238 [19:43<00:00,  2.74it/s, train_loss=0.0127, train_mae=7.85°]


Epoch 32 → Val MAE = 21.44°   (best 20.14°)


Epoch 33: 100%|██████████| 3238/3238 [19:42<00:00,  2.74it/s, train_loss=0.0128, train_mae=7.69°]


...

Epoch 49: 100%|██████████| 3238/3238 [19:38<00:00,  2.75it/s, train_loss=0.0062, train_mae=5.21°]


Epoch 49 → Val MAE = 18.90°   (best 19.87°)
  ✔ New best model saved.


Epoch 50: 100%|██████████| 3238/3238 [19:39<00:00,  2.75it/s, train_loss=0.0060, train_mae=5.07°]


Epoch 50 → Val MAE = 21.79°   (best 18.90°)
Continued training complete. Best Val MAE = 18.90°


: 